<a href="https://colab.research.google.com/github/LifelessPROgrammer/SEM3ML/blob/colabs/5A%5DWrite_a_program_to_demonstrate_the_working_of_the_decision_tree_based_ID3_algorithm_Use_an_appropriate_data_set_for_building_the_decision_tree_and_apply_this_knowledge_to_classify_a_new_sample_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import math
import numpy as np

In [3]:
class Node:
    def __init__(self):
        self.children = []
        self.value = ""
        self.isLeaf = False
        self.pred = ""

In [5]:
def entropy(examples):
    pos = 0.0
    neg = 0.0
    for _, row in examples.iterrows():
        if row["answer"] == "yes":
            pos += 1
        else:
            neg += 1
    if pos == 0.0 or neg == 0.0:
        return 0.0
    else:
        p = pos / (pos + neg)
        n = neg / (pos + neg)
        return -(p * math.log(p, 2) + n * math.log(n, 2))

In [6]:
def info_gain(examples, attr):
    uniq = np.unique(examples[attr])
    gain = entropy(examples)
    for u in uniq:
        subdata = examples[examples[attr] == u]
        sub_e = entropy(subdata)
        gain -= (len(subdata) / len(examples)) * sub_e
    return gain

In [7]:
def ID3(examples, attrs):
    root = Node()
    max_gain = 0
    max_feat = ""
    for feature in attrs:
        gain = info_gain(examples, feature)
        if gain > max_gain:
            max_gain = gain
            max_feat = feature

    print("Max feature:", max_feat)
    print("Attrs before removal:", attrs)

    # Create a new list excluding max_feat
    new_attrs = [feat for feat in attrs if feat != max_feat]

    print("Attrs after removal:", new_attrs)

    root.value = max_feat
    uniq = np.unique(examples[max_feat])
    for u in uniq:
        subdata = examples[examples[max_feat] == u]
        if entropy(subdata) == 0.0:
            newNode = Node()
            newNode.isLeaf = True
            newNode.value = u
            newNode.pred = np.unique(subdata["answer"])
            root.children.append(newNode)
        else:
            dummyNode = Node()
            dummyNode.value = u
            child = ID3(subdata, new_attrs)
            dummyNode.children.append(child)
            root.children.append(dummyNode)
    return root

In [8]:
def printTree(root: Node, depth=0):
    for i in range(depth):
        print("\t", end="")
    print(root.value, end="")
    if root.isLeaf:
        print(" -> ", root.pred)
        print()
    for child in root.children:
        printTree(child, depth + 1)

In [9]:
def classify(root: Node, new):
    for child in root.children:
        if child.value == new[root.value]:
            if child.isLeaf:
                print("Predicted Label for new example", new," is:", child.pred)
                return
            else:
                classify(child.children[0], new)

In [11]:
data = pd.read_csv("/content/ID3 algorithm (1).csv")
features = [feat for feat in data]
features.remove("answer")

In [12]:
root = ID3(data, features)
print("Decision Tree is:")
printTree(root)
print("------------------")
new = {"outlook": "sunny", "temperature": "hot", "humidity": "normal", "wind": "strong"}
classify(root, new)

Max feature: outlook
Attrs before removal: ['outlook', 'temperature', 'humidity', 'wind']
Attrs after removal: ['temperature', 'humidity', 'wind']
Max feature: wind
Attrs before removal: ['temperature', 'humidity', 'wind']
Attrs after removal: ['temperature', 'humidity']
Max feature: humidity
Attrs before removal: ['temperature', 'humidity', 'wind']
Attrs after removal: ['temperature', 'wind']
Decision Tree is:
outlook	overcast ->  ['yes']

	rain		wind			strong ->  ['no']

			weak ->  ['yes']

	sunny		humidity			high ->  ['no']

			normal ->  ['yes']

------------------
Predicted Label for new example {'outlook': 'sunny', 'temperature': 'hot', 'humidity': 'normal', 'wind': 'strong'}  is: ['yes']
